# Monte Carlo Simulations

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as s

**Price a regular call and put option using monte carlo simulation and compare with Black Scholes Price. Compute the confidence interval of the estimate. Use appropraite variance reduction procedures to reduce the standard error.**

In [2]:
S0 = 120
T = 0.5
r = 0.03
vol = 0.3
K = 130

_1. Option Price using Black Scholes Model_

In [3]:
def blackscholes(S0,T,K,r,vol,call=1):
    d1 = (np.log(S0/K)+(r+0.5*vol**2)*(T))/(vol*T**0.5)
    d2 = d1-(vol*T**0.5)
    if call==1:
        price = S0*s.stats.norm.cdf(d1)-K*np.exp(-r*T)*s.stats.norm.cdf(d2)
    else:
        price = -S0*s.stats.norm.cdf(-d1)+K*np.exp(-r*T)*s.stats.norm.cdf(-d2)
    return price

In [4]:
print(f'The Black Scholes price of the call option is {round(blackscholes(S0,T,K,r,vol,call=1),3)}')
print(f'The Black Scholes price of the put option is {round(blackscholes(S0,T,K,r,vol,call=0),3)}')

The Black Scholes price of the call option is 6.931
The Black Scholes price of the put option is 14.996


_2. Option Price using Monte Carlo Simulation_

In [5]:
def normal_samples(n):
    return [np.random.normal() for i in range(n)]

In [6]:
def stock_path(S0,m,s,t,phi):
    return S0*np.exp((m-0.5*s**2)*t+s*phi*(t)**0.5)

In [7]:
def payoff(stock_paths,K,call=1):
    if call==1:
        payoffs =np.maximum(np.array(stock_paths)-K,0)
    else:
        payoffs =np.maximum(K-np.array(stock_paths),0)
    return payoffs

In [8]:
x = normal_samples(100000)

In [9]:
c_payoffs = payoff([stock_path(S0,r,vol,T,x[i]) for i in range(10000)],K,call=1)
p_payoffs = payoff([stock_path(S0,r,vol,T,x[i]) for i in range(10000)],K,call=0)

In [10]:
c = np.mean(c_payoffs)*np.exp(-r*T)
p =np.mean(p_payoffs)*np.exp(-r*T)

print(f'Call Price: {round(c,3)}')
print(f'Put Price: {round(p,3)}')

Call Price: 6.878
Put Price: 15.207


In [11]:
SE_c =np.std(c_payoffs)*np.exp(-r*T)/(len(c_payoffs)**0.5)
SE_p =np.std(p_payoffs)*np.exp(-r*T)/(len(p_payoffs)**0.5)

print(f'S.E of Call Price: {round(SE_c,3)}')
print(f'S.E of Put Price: {round(SE_p,3)}')

S.E of Call Price: 0.142
S.E of Put Price: 0.16


In [12]:
def confidence_interval(m,sd,level =0.95):
    l = m+sd*s.stats.norm.ppf((1-level)/2)
    u = m+sd*s.stats.norm.ppf(1-(1-level)/2)
    return round(l,2),round(u,2)

In [13]:
print(f'The 95% confidence interval for call price is {confidence_interval(c,SE_c,level =0.95)}')
print(f'The 95% confidence interval for put price is {confidence_interval(p,SE_p,level =0.95)}')

The 95% confidence interval for call price is (6.6, 7.16)
The 95% confidence interval for put price is (14.89, 15.52)


The estimate of option prices found using montecarlo simulation are close to the black scholes price. But the standard error is large. For example, 5% of the time the call option price estimated is going to be below 5.93 which is about $1 lower than the true price. Using variance reduction procedures we might be able to reduce the standard error.

_3. Using antithetic Variable Technique_

In [14]:
x_1 = normal_samples(100000)
x_2 = np.array(x_1)*-1

In [15]:
c_payoffs_1 = payoff([stock_path(S0,r,vol,T,x_1[i]) for i in range(10000)],K,call=1)
p_payoffs_1 = payoff([stock_path(S0,r,vol,T,x_1[i]) for i in range(10000)],K,call=0)

c_payoffs_2 = payoff([stock_path(S0,r,vol,T,x_2[i]) for i in range(10000)],K,call=1)
p_payoffs_2 = payoff([stock_path(S0,r,vol,T,x_2[i]) for i in range(10000)],K,call=0)

c_payoffs = (c_payoffs_1+c_payoffs_2)/2
p_payoffs = (p_payoffs_1+p_payoffs_2)/2


In [16]:
c = np.mean(c_payoffs)*np.exp(-r*T)
p =np.mean(p_payoffs)*np.exp(-r*T)

print(f'Call Price: {round(c,3)}')
print(f'Put Price: {round(p,3)}')

Call Price: 6.903
Put Price: 14.979


In [17]:
SE_c =np.std(c_payoffs)*np.exp(-r*T)/(len(c_payoffs)**0.5)
SE_p =np.std(p_payoffs)*np.exp(-r*T)/(len(p_payoffs)**0.5)

print(f'S.E of Call Price: {round(SE_c,3)}')
print(f'S.E of Put Price: {round(SE_p,3)}')

S.E of Call Price: 0.088
S.E of Put Price: 0.051


In [18]:
print('The reduction in SE is about:')
print(f'Calls : {round(0.278/0.442*100,2)}%')
print(f'Puts : {round(0.162/0.486*100,2)}%')

The reduction in SE is about:
Calls : 62.9%
Puts : 33.33%


In [19]:
print(f'The 95% confidence interval for call price is {confidence_interval(c,SE_c,level =0.95)}')
print(f'The 95% confidence interval for put price is {confidence_interval(p,SE_p,level =0.95)}')

The 95% confidence interval for call price is (6.73, 7.08)
The 95% confidence interval for put price is (14.88, 15.08)


We can see that the confidence interval for the estimate has become narrower